LGBMをOptunaでチューニング... 
fExp上位(=fImp上位)のFeatureを順に削っていき順次モデル作成


In [1]:
import seaborn as sns
import csv
import optuna
import optuna.integration.lightgbm as lgb
import itertools, logging, joblib
NOTE_NAME = "lgbm2"

In [2]:
ROUND = "237"
TOURNAMENT_NAME = "kazutsugi"
TARGET_NAME = f"target_{TOURNAMENT_NAME}"
PREDICTION_NAME = f"prediction_{TOURNAMENT_NAME}"
N_FEATURE = 310

In [3]:
# signal
signals = pd.read_csv(f"{NOTE_NAME}/lgbm2_signals_train.csv", index_col=["id"])
val_signals = pd.read_csv(f"{NOTE_NAME}/lgbm2_signals_val.csv", index_col=["id"])

In [4]:
# set logger
logger = logging.getLogger()  # root logger
fh = logging.FileHandler(f'{NOTE_NAME}/train4.log')
fh.setFormatter(logging.Formatter('%(asctime)s|%(message)s', '%Y-%m-%d %H:%M:%S'))
logger.addHandler(fh)

optuna.logging.enable_propagation()  # Propagate logs to the root logger.
optuna.logging.disable_default_handler()  # Stop showing logs in sys.stderr.

In [ ]:
# set train data
train_lgb = lgb.Dataset(signals.iloc[:, :6], label=signals[TARGET_NAME])
val_lgb = lgb.Dataset(val_signals.dropna().iloc[:, :6], val_signals.dropna()[TARGET_NAME])

## fit setting 
default_params = {
    'objective': 'regression',
    'metric': 'l2',
}

best_params, tuning_history = dict(), list()
model = lgb.LightGBMTuner(
    default_params, train_lgb, valid_sets=val_lgb, early_stopping_rounds=40,
    time_budget=3600*24, verbose_eval=False
)

# fit
model.run()

# save
joblib.dump(model.best_booster, f"{NOTE_NAME}/ensemble_lgb.pkl")


feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

---

In [7]:
model.best_params

{'objective': 'regression',
 'metric': 'l2',
 'feature_pre_filter': False,
 'lambda_l1': 0.0,
 'lambda_l2': 0.0,
 'num_leaves': 114,
 'feature_fraction': 0.4,
 'bagging_fraction': 0.9995331976144334,
 'bagging_freq': 7,
 'min_child_samples': 20}

In [19]:
pd.DataFrame(model.best_booster.predict(signals.iloc[:, :6]), index=signals.index, columns=["pred"]).to_csv(f"{NOTE_NAME}/ensemble_pred.csv")

/home/tarai/.pyenv/versions/anaconda3-5.3.0/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: best_booster has been deprecated in v1.4.0. This feature will be removed in v3.0.0.
  """Entry point for launching an IPython kernel.


In [20]:
pd.DataFrame(model.best_booster.predict(val_signals.iloc[:, :6]), index=val_signals.index, columns=["pred"]).to_csv(f"{NOTE_NAME}/ensemble_pred_val.csv")

/home/tarai/.pyenv/versions/anaconda3-5.3.0/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: best_booster has been deprecated in v1.4.0. This feature will be removed in v3.0.0.
  """Entry point for launching an IPython kernel.
